# Lab 2: Sequence Alignment

### Name: Your Name Here (Your netid here)

### Due March 18, 2020 11:59 PM

#### Preamble (Don't change this)

## Important Instructions - 

1. You are not allowed to use any built-in libraries for calculating Smith-Waterman alignment/score.
2. Please implement all the *graded functions* in main.py file. Do not change function names in main.py.
3. Please read the description of every graded function very carefully. The description clearly states what is the expectation of each graded function. 
4. After some graded functions, there is a cell which you can run and see if the expected output matches the output you are getting. 
5. The expected output provided is just a way for you to assess the correctness of your code. The code will be tested on several other cases as well.

In [221]:
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm 
import numpy as np
import pickle

# Sequence Alignment

In this lab, we will look into performing sequence alignment between genomic sequences.
As we discussed in class, this is a key computational task in genomics.
In particular, sequence alignment is used in the following two scenarios:
* When we sequence the DNA of an organism that we have never sequenced before, we need to align the reads to each other in order to recover the entire genome.
* When we sequence the DNA of an organism for which a reference genome is available (e.g., humans), we need to align the reads to the reference genome.

Abstractly, in the sequence alignment problem, we are given two sequences $x$ and $y$, and we want to place gaps (represented by ‘-’) in $x$ and $y$ so that the resulting sequences “line up well”.
For example, if $x = AGGCTAGTT$ and $y = AGCGAAGTTT$, a "good" alignment is 

```
AGGC-TA-GTT-
AG-CG-AAGTTT
```

As we discussed in class, the Smith-Waterman algorithm assigns scores/penalties to matches, mismatches, and gaps gaps, and then computes the alignment between the two sequences that maximizes the total score.

The Smith-Waterman algorithm performs *local* sequence alignment. This means that we are looking for a substring of x and a substring of y with the largest possible alignment score.
For example, if our scores are +1 for match, -1 for mismatch, -1 for gap and we want to align $x = CCCCGATTACAGGGG$ and $y = GGGGGATACACCCC$, then the best possible local alignment is

```
GATTACA
GAT_ACA
```

which has score 6-1=5. Notice that the gaps in the beginning and in the end don't 


### PacBio data

We will start with the same PacBio data from Lab 1. 
PacBio reads are typically long, and aligning them can be challenging in practice.

In [113]:
#reading PacBio data
with open('dna_reads_pac-bio.data', 'rb') as filehandle:
    dna_reads_pac=pickle.load(filehandle)

In [171]:
%run main.py

The following line creates an object from the class in *main.py*. **Do not change the class name and function headers!**

In [172]:
module = Lab2()

## Graded Function 1: smith_waterman_alignment  (10 marks)

Purpose - To perform local sequence alignment between two DNA sequences and identify sequence similarity using the Smith-Waterman algorithm. You should calculate alignment score between every two points in the sequences and record the maximum score.

Input - two sequences and a dictionary with penalties for match, mismatch and gap (e.g., `penalties={'match':1,'mismatch':-1,'gap':-1}`)

Output - an integer value which is the maximum smith waterman alignment score

In [116]:
penalties={'match':1,'mismatch':-1,'gap':-1}

In [117]:
# Note this may take some time to compute
print(module.smith_waterman_alignment(dna_reads_pac[0],dna_reads_pac[1],penalties))

593.0


### Expected Output - 

593

As you noticed, finding the optimal alignment between two long PacBio reads takes a while. 
Imagine doing this for hundreds of thousands of reads!
Some of the indexing techniques that we will explore later in this lab can be used in practice to accelerate this process.

## Graded Function 2: print_smith_waterman_alignment  (10 marks)

Purpose - To perform local sequence alignment between two DNA sequences and return the resulting alignment in a nice fashion, like:

```
AGGC-TA-GTT-
AG-CG-AAGTTT
```

Input - two sequences and a dictionary with penalities for match, mismatch and gap

Output - return a tuple with two strings showing the two sequences with '-' representing the gaps

In [111]:
DIAG = 0
LEFT = 1
UP = 2

# print(s1, s2)
# print(inspect.getsource(inspect.currentframe().f_back))

def function(s1, s2, penalties):    
    dp = np.zeros((len(s1)+1, len(s2)+1))
    dirs = np.zeros((len(s1)+1, len(s2)+1))

    for i in range(1, dp.shape[0]):
        for j in range(1, dp.shape[1]):
            if s1[i-1] == s2[j-1]:
                diag = (dp[i-1, j-1] + penalties['match'], 0)
            else:
                diag = (dp[i-1, j-1] + penalties['mismatch'], 0)
            left = (dp[i, j-1] + penalties['gap'], 1)
            up = (dp[i-1, j] + penalties['gap'], 2)
            dp[i, j], dirs[i, j] = max((0, DIAG), left, up, diag)
    return dp, dirs

def get_alignment_matrix(s1, s2, penalties):
    dp = np.zeros((len(s1)+1, len(s2)+1))
    dirs = np.zeros((len(s1)+1, len(s2)+1), dtype=np.ubyte)
    m = penalties['match']
    s = -penalties['mismatch']
    d = -penalties['gap']
    for i in range(1, dp.shape[0]):
        x = s1[i-1]
        for j in range(1, dp.shape[1]):
            y = s2[j-1]
            if x == y:
                diag = (dp[i-1, j-1] + penalties['match'], DIAG)
            else:
                diag = (dp[i-1, j-1] - s, DIAG)
            left = (dp[i, j-1] - d, LEFT)
            up = (dp[i-1, j] - d, UP)
            dp[i, j], dirs[i, j] = max((0, DIAG), left, up, diag)
    return dp, dirs

def print_smith_waterman_alignment(s1,s2,penalties):
    dp = np.zeros((len(s1)+1, len(s2)+1))
    dirs = np.zeros((len(s1)+1, len(s2)+1))
    for i in range(1, dp.shape[0]):
        for j in range(1, dp.shape[1]):
            if s1[i-1] == s2[j-1]:
                diag = (dp[i-1, j-1] + penalties['match'], 0)
            else:
                diag = (dp[i-1, j-1] + penalties['mismatch'], 0)
            left = (dp[i, j-1] + penalties['gap'], 1)
            up = (dp[i-1, j] + penalties['gap'], 2)
            dp[i, j], dirs[i, j] = max((0, DIAG), left, up, diag)
    maxScoreIndex =  tuple(dp.shape - np.array(np.unravel_index(np.argmax(np.flip(dp)), dp.shape)) - 1)
    currScore = dp[maxScoreIndex]
    string1 = ''
    string2 = ''
    while currScore != 0:
        if dirs[maxScoreIndex] == 0:
            string1 = s1[maxScoreIndex[0]-1] + string1
            string2 = s2[maxScoreIndex[1]-1] + string2
            maxScoreIndex = (maxScoreIndex[0]-1, maxScoreIndex[1]-1)
        elif dirs[maxScoreIndex] == 1:
            string1 = '-' + string1
            string2 = s2[maxScoreIndex[1]-1] + string2
            maxScoreIndex = (maxScoreIndex[0], maxScoreIndex[1]-1)
        elif dirs[maxScoreIndex] == 2:
            string1 = s1[maxScoreIndex[0]-1] + string1
            string2 = '-' + string2
            maxScoreIndex = (maxScoreIndex[0]-1, maxScoreIndex[1])
        currScore = dp[maxScoreIndex]
    return (string1, string2)
    
    
    
alignment = print_smith_waterman_alignment("CTCGCAATATGCTAGCAGC","GATCGCAATCTGCAGTCCG",penalties)
print(alignment)
if alignment==('TCGCAATATGCTAG', 'TCGCAATCTGC-AG') or alignment==('TCGCAATATGCTAG-CAG', 'TCGCAATCTGC-AGTCCG') or alignment==('TCGCAATATGCTAG-C', 'TCGCAATCTGC-AGTC'):
    print("True")
else :
    print("False")
    
x = "MISPEL"
y = "MISSPELL"
alignment = print_smith_waterman_alignment(x,y,penalties)
print(alignment)
    
    
x="CCACACCACACCCACACACCCACACACCACACCACACACCCACCACACCCACACACACACATCCTAACAACTACCCTAACACAGCCCTAATCTAACCCTG"
y='CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTG'
alignment=print_smith_waterman_alignment(x,y,penalties)
print(alignment)
if alignment==('CCACACCACACCCACACACCCACACACCACACCACACACC-CACCACACCCACACACACACATCCTAACAACTACCCTAACACAGCCCTAATCTAACCCTG',
 'CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAAC-ACTACCCTAACACAGCCCTAATCTAACCCTG') :
    print("True")
else :
    print("False")

('TCGCAATATGCTAG-CAG', 'TCGCAATCTGC-AGTCCG')
True
('MIS-PEL', 'MISSPEL')
('CCACACCACACCCACACACCCACACACCACACCACACACC-CACCACACCCACACACACACATCCTAACAACTACCCTAACACAGCCCTAATCTAACCCTG', 'CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACA-CTACCCTAACACAGCCCTAATCTAACCCTG')
False


In [120]:
%run main.py
module = Lab2()
x = "MISPEL"
y = "MISSPELL"
print(module.print_smith_waterman_alignment(x,y,penalties))

alignment = print_smith_waterman_alignment("CTCGCAATATGCTAGCAGC","GATCGCAATCTGCAGTCCG",penalties)
print(alignment)
if alignment==('TCGCAATATGCTAG', 'TCGCAATCTGC-AG') or alignment==('TCGCAATATGCTAG-CAG', 'TCGCAATCTGC-AGTCCG') or alignment==('TCGCAATATGCTAG-C', 'TCGCAATCTGC-AGTC'):
    print("True")
else :
    print("False")

('MIS-PEL', 'MISSPEL')
('TCGCAATATGCTAG-CAG', 'TCGCAATCTGC-AGTCCG')
True


### Expected Output - 

``('MI-SPEL', 'MISSPEL')``

or 

``('MIS-PEL', 'MISSPEL')``

# Aligning reads to a (long) genome

While the Smith-Waterman algorithm can provide local alignments between two sequences of arbitrary lengths, it is too slow to be used to align reads to a long genome.
As we discussed in class, when we are trying to align reads to a long genome, we typically rely on an indexing scheme (based on hash functions, or a Python dictionary) to quickly identify matches.

We will consider two genome files.
The first one is a short fake genome in the file "fakegenome.fasta".

The second one is the *Saccharomyces cerevisiae* (Brewer's yeast) genome.
The *S. cerevisiae* genome was the first eukaryotic genome to be fully sequenced.
It contains 16 chromosomes for a total genome length of about 12 million base-pairs.

In [123]:
fakegenome_file=""
with open("fakegenome.fasta") as file:
    fakegenome_file=file.read()

saccha_file=""
with open("saccha.fasta") as file:
    saccha_file=file.read()

In [3]:
# let's print the fakegenome file and the beginning of the S. cerevisiae file:

print(fakegenome_file)
print()
print(saccha_file[:300])

>chr1
GATTACA
>chr2
CAGATTTACACATACA
>chr3
CACACACA


>chr1
CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACA
CATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTT
ACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAAC
CACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATC
CAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTA


Notice that the chromosomes are separated by a line that only contains ">chrN", where N is the chromosome number

## Graded Function 3 : find_exact_matches(list_of_reads,genome_file)

Pupose - To check whether each of the reads in list_of_reads is present (exactly) somewhere in the genome and, if so, return the location. The location should be specified as "chr2:120000" (for a read that starts at position 120000 of chromosome 2)

Input - list of reads **of the same length** and a genome fasta file (converted into a single string)

Output - a list with the same length as list_of_reads, where the ith element is a list of all locations (starting positions) in the genome where the ith read appears. The starting positions should be specified using the "chr2:120000" format

Note: Avoid using Python packages and built-in functions to do search operations (such as the find function). The goal of this problem is for you to practice using Python dictionaries to build a genome index that can help finding matches quickly.

Note: Genomic locations should be spaced using 1-based indexing. For example, the first position of chromosome 3 should be specified as ``chr3:1`` (and not ``chr3:0``)

In [25]:
list_of_fake_reads = ['GATT','TACA','CACA']

import re

def find_exact_matches(list_of_reads,genome):
    dic = {}
    substringSize = len(list_of_reads[0])
    x = genome.split("\n")
    gene = ""
    currChar = x[0]
    for i in range(1,len(x)):
        if "chr" in x[i]:
            dic[currChar] = gene
            gene = ""
            currChar = x[i]
        else:
            gene+= x[i]
    dic[currChar] = gene
    
    # print(dic)
    
#     res = [i.start() for i in re.finditer('CTGTATGGCTATACGATTATGTGGGCTACCAACAGATTGGTCACTTTCCT', dic['>chr15'])] 
#     print(res)
    
    charDicts = []
    for k,v in dic.items():
        tempDic = {}
        for i in range(len(v)-substringSize+1):
            if v[i:i+substringSize] in tempDic:
                tempDic[v[i:i+substringSize]].append(i+1)
            else:
                tempDic[v[i:i+substringSize]] = []
                tempDic[v[i:i+substringSize]].append(i+1)
                
        charDicts.append(tempDic)
    
    answer = []
    for pattern in list_of_reads:
        tempAnswer = []
        for i in range(len(charDicts)):
            z = 'chr'+str((i+1))
            if pattern in charDicts[i]:
                startPoints = charDicts[i][pattern]
                for j in startPoints:
                    pos = ""
                    pos+=z
                    pos+=":"
                    pos+=str(j)
                    tempAnswer.append(pos)
        answer.append(tempAnswer)
                    
        
    return answer
     
# print(find_exact_matches(list_of_fake_reads,fakegenome_file))
read0 = "CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACC"
read1 = "CACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAG"
read2 = "CTCGCTGTCACTCCTTACCCGGCTTTCTGACCGAAATTAAAAAAAAAAAA"
read3 = "TTTAAACTTACGATTATGTGATTTGATGAGGTCAATCAACAGATTAACCA"
read4 = "CTGTATGGCTATACGATTATGTGGGCTACCAACAGATTGGTCACTTTCCT"
read5 = "GGGTCCGATGTTGGATTGAAATCCCAAGGTGCTATTTCTATATTTATATA"
list_of_reads = [read0,read1,read2,read3,read4]
print(find_exact_matches(list_of_reads,saccha_file))

[]
None


In [21]:
list_of_fake_reads = ['GATT','TACA','CACA']
print(module.find_exact_matches(list_of_fake_reads,fakegenome_file))

print(saccha_file[753362:753413])

[['chr1:1', 'chr2:3'], ['chr1:4', 'chr2:7', 'chr2:13'], ['chr2:9', 'chr3:1', 'chr3:3', 'chr3:5']]
GCCTCCGAATCATTAATGCATTGTTTGGAAAGGCGGTTGCCTCAGGCAGAG


### Expected Output - 

``[['chr1:1', 'chr2:3'], ['chr1:4', 'chr2:7', 'chr2:13'], ['chr2:9', 'chr3:1', 'chr3:3', 'chr3:5']]``

In [22]:
read0 = "CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACC"
read1 = "CACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAG"
read2 = "CTCGCTGTCACTCCTTACCCGGCTTTCTGACCGAAATTAAAAAAAAAAAA"
read3 = "TTTAAACTTACGATTATGTGATTTGATGAGGTCAATCAACAGATTAACCA"
read4 = "CTGTATGGCTATACGATTATGTGGGCTACCAACAGATTGGTCACTTTCCT"
read5 = "GGGTCCGATGTTGGATTGAAATCCCAAGGTGCTATTTCTATATTTATATA"
list_of_reads = [read0,read1,read2,read3,read4]

print(module.find_exact_matches(list_of_reads,saccha_file))

[['chr1:1'], ['chr1:35'], ['chr8:56', 'chr13:73'], ['chr2:753363'], []]


# Aligning reads with errors/mutations to a (long) genome

When the reads may have discrepancies with respect to the reference genome (which could be sequencing errors or mutations), we need to be more clever with our indexing.

In the following, we will use the same two genome files (fakegenome_file and saccha_file) from above, but consider reads with errors/mutations.

## Graded Function 4 : find_approximate_matches(list_of_reads,genome_file)

Purpose - To return the locations in the genome file which have the highest Smith-Waterman alignment score for each of the reads in list_of_reads. All reads in the list will have the same length, say $L$. For each read, your goal is to find a segment of length $L$ in the genome with the largest Smith-Waterman alignment score with the read. 

Notice that simply running Smith-Waterman between a read and every length-$L$ segment in the genome is impractical (and will take too long). Instead you should use an indexing scheme, based on Python dictionaries, to first identify candidate locations in the genome, and then use the Smith-Waterman algorithm to find the alignment score.

For Smith-Waterman, you should use penalties={'match':1,'mismatch':-1,'gap':-1}.

Input - list of reads of the same length and a genome fasta file (converted into a single string)

Output - a list with the same length as list_of_reads, where the ith element is a list of all locations (starting positions) in the genome which have the highest Smith-Waterman alignment score with the ith read in list_of_reads

Note: The location should be specified as "chr2:120000" (for the length-$L$ segment that starts at position 120000 of chromosome 2). As in Graded function 3, you should specify the position using 1-based indexing; i.e., the chromosome starts at position 1, not at 0).

Note: there can be more than one position for a particular read which have the same highest score. You should include all of them as a list.

**Hint:** For all cases that we will be checking, you can choose the substring length (for the genome index) to be $k = L/4$. We will only use lengths $L$ that are divisible by 4.

In [222]:
print(fakegenome_file)

>chr1
GATTACA
>chr2
CAGATTTACACATACA
>chr3
CACACACA



In [211]:
def find_approximate_matches(list_of_reads,genome):
    penalties={'match':1,'mismatch':-1,'gap':-1}
    dic = {}
    substringSize =int(len(list_of_reads[0])/4)
    x = genome.split("\n")
    gene = ""
    currChar = x[0]
    for i in range(1,len(x)):
        if "chr" in x[i]:
            dic[currChar] = gene
            gene = ""
            currChar = x[i]
        else:
            gene+= x[i]
    dic[currChar] = gene
        
        # print(dic)
        
    charDicts = []
    for k,v in dic.items():
        tempDic = {}
        for i in range(len(v)-substringSize+1):
            if v[i:i+substringSize] in tempDic:
                tempDic[v[i:i+substringSize]].append(i+1)
            else:
                tempDic[v[i:i+substringSize]] = []
                tempDic[v[i:i+substringSize]].append(i+1)
                    
        charDicts.append(tempDic)
        
        # print(charDicts)
    finalAnswer = []
    for i in list_of_reads:
        answerDic = {}
        for j in range(len(i)-substringSize):
            string = i[j:j+substringSize]
                # print(i[j:j+2])
            for k in range(len(charDicts)):
                dicKey = ">chr"+str(k+1)
                dicString = dic[dicKey]
                    
                if string in charDicts[k]:
                    startPoints = charDicts[k][string]
                    maxStartingPoint = float('-inf')
                    maxStartingScore = float('-inf')
                    for s in startPoints:
                        start  = s-j
                        end = s-j+len(i)
                        if end <= len(dicString):
                            score = module.smith_waterman_alignment(dicString[start: end+1],i,penalties)
                            if score > maxStartingScore:
                                maxStartingPoint = start
                                maxStartingScore = score
                    if maxStartingPoint != float('-inf') and  maxStartingScore != float('-inf'):
                        if maxStartingScore in answerDic:
                            tempDic = answerDic[maxStartingScore]
                            if dicKey in tempDic:
                                if maxStartingPoint>tempDic[dicKey]:
                                    tempDic[dicKey] = maxStartingPoint+1
                            else:
                                tempDic[dicKey] = maxStartingPoint+1
                                
                            answerDic[maxStartingScore] = tempDic
                        else:
                            tempDic = {dicKey:maxStartingPoint+1}
                            answerDic[maxStartingScore] = tempDic
        print(answerDic)
                                
#         maxScore = max(answerDic, key=int)
#         values = answerDic[maxScore]
#         tempAnswer = []
#         for k,v in values.items():
#             ans = k[1:]+":"+str(v-1)
#             tempAnswer.append(ans)
#         finalAnswer.append(tempAnswer)
#     return finalAnswer
    
                        
        
    
    
    
    
print(find_approximate_matches(["GATTACAT","CACAAACA"],fakegenome_file))
# print(find_approximate_matches(["CACAAACA"],fakegenome_file))

{5.0: {'>chr2': 4}, 3.0: {'>chr3': 1}}
{3.0: {'>chr2': 2}, 6.0: {'>chr2': 8}, 1.0: {'>chr3': 0}, 4.0: {'>chr2': 4}}
None


In [ ]:
['TGCAGATTGCTCCTACGCTTGACAATGTCGGATCCGATACCGATCTGATTCATATCGATACAGTTAGTGCCATTAACGAGCAATTTCTAAGACTGCACTG', 'ACGTAAAAAATGTAGCAGACTCGATCTCCTCTTCTGATGAAATCCTAGTTCCTTCGAGACTCGCTGATGTTACGCTAGCATTCATGGAGGAGAATGACGC', 'AAGTGGAAAGAAAGAAGGGTGACAAGTTCGTCGCTTGTTTCACAAGATTACCAACGCCAGCCATATTGTAACATAGATGTATAACTAGAACAATTTACCA', 'CCACACCACACCCACACACCCACACACCACACCACACACCCACCACACCCACACACACACATCCTAACAACTACCCTAACACAGCCCTAATCTAACCCTG']
    @partial_credit(10)
    def test_find_approximate_matches(self,set_score=None):
        total_score=0
        wrong_cases=0
        matches=self.module.find_approximate_matches(["GATTACAT","CACAAACA"],self.fakegenome_file)
        s=set()
        for match in matches :
            match.sort()
            s.add(tuple(match))
        set1=tuple(['chr2:3'])
        set2=tuple(['chr2:9', 'chr3:1'])
        s1=set([set2,set1])
        testValue=s==s1
        if testValue :
            total_score+=2
        else :
            wrong_cases+=1
            
        read0 = "TGCAGATTGCTCCTACGCTTGACAATGTCGGATCCGATACCGATCTGATTCATATCGATACAGTTAGTGCCATTAACGAGCAATTTCTAAGACTGCACTG"
        read1 = "ACGTAAAAAATGTAGCAGACTCGATCTCCTCTTCTGATGAAATCCTAGTTCCTTCGAGACTCGCTGATGTTACGCTAGCATTCATGGAGGAGAATGACGC"
        read2 = "AAGTGGAAAGAAAGAAGGGTGACAAGTTCGTCGCTTGTTTCACAAGATTACCAACGCCAGCCATATTGTAACATAGATGTATAACTAGAACAATTTACCA"
        read3 = "CCACACCACACCCACACACCCACACACCACACCACACACCCACCACACCCACACACACACATCCTAACAACTACCCTAACACAGCCCTAATCTAACCCTG"
        list_of_reads = [read0,read1,read2,read3]
        matches=self.module.find_approximate_matches(list_of_reads,self.saccha_file)
        s=set()
        for match in matches :
            match.sort()
            s.add(tuple(match))
        set1=tuple(['chr14:11909', 'chr6:10002'])
        set2=tuple(['chr12:30122'])
        set3=tuple(['chr4:100351'])
        set4=tuple(['chr1:1'])
        s1=set([set4,set3,set2,set1])
        testValue=s==s1
        if testValue :
            total_score+=8
        else :
            wrong_cases+=1
        print(str(wrong_cases) + " test cases failed")
        set_score(total_score)

In [229]:
from collections import defaultdict 
def func(list_of_reads,genome):
    penalties={'match':1,'mismatch':-1,'gap':-1}
    first_split = genome.split('>')[1:]
    second_split = [x.split('\n') for x in first_split]
    chr_str = {x[0]:''.join(x[1:]) for x in second_split}
    k = len(list_of_reads[0])//4
    L = len(list_of_reads[0])
    chr_substr = {}
    for x, y in chr_str.items():
        substrings = [(y[i:i+k], i) for i in range(0, len(y)-k+1)]
        chr_substr[x] = substrings
    substr_chr = defaultdict(list)
    for x, y in chr_substr.items():
        for s, loc in y:
            substr_chr[s].append((x, loc))
    ans = []
    for read in list_of_reads:
        read_substrings = [(read[i:i+k], i) for i in range(0, len(read)-k+1)]
        aligned_locs_scores_set = set()
        for substring, read_index in read_substrings:
            if substring not in substr_chr:
                continue
            locs = substr_chr[substring]
            for chrom, i in locs:
                genome = chr_str[chrom]
                if (len(genome) < L):
                    continue
                start = max(0, i - read_index)
                stop = min(len(genome), start + L)
                s2 = genome[start:stop]
                s1 = read
                score = module.smith_waterman_alignment(s1, s2, penalties)
                aligned_locs_scores_set.add((score, '{}:{}'.format(chrom, start+1)))
        if not aligned_locs_scores_set:
            ans.append([])
            continue
        aligned_locs_scores = list(aligned_locs_scores_set)
        max_score = max(aligned_locs_scores)[0]
        index_locations = [x[1] for x in aligned_locs_scores if x[0] == max_score]
        ans.append(index_locations)

    return ans

In [230]:
%run main.py
module = Lab2()
print(func(["GATTACAT","CACAAACA"],fakegenome_file))

[['chr2:3'], ['chr2:9', 'chr3:1']]


### Expected Output - 

``[['chr2:3'], ['chr2:9', 'chr3:1']]``

In [208]:
read0 = "TGCAGATTGCTCCTACGCTTGACAATGTCGGATCCGATACCGATCTGATTCATATCGATACAGTTAGTGCCATTAACGAGCAATTTCTAAGACTGCACTG"
read1 = "ACGTAAAAAATGTAGCAGACTCGATCTCCTCTTCTGATGAAATCCTAGTTCCTTCGAGACTCGCTGATGTTACGCTAGCATTCATGGAGGAGAATGACGC"
read2 = "AAGTGGAAAGAAAGAAGGGTGACAAGTTCGTCGCTTGTTTCACAAGATTACCAACGCCAGCCATATTGTAACATAGATGTATAACTAGAACAATTTACCA"
read3 = "CCACACCACACCCACACACCCACACACCACACCACACACCCACCACACCCACACACACACATCCTAACAACTACCCTAACACAGCCCTAATCTAACCCTG"

list_of_reads = [read0,read1,read2,read3]
#list_of_reads = [read1]

print(module.find_approximate_matches(list_of_reads,saccha_file)) 

[['chr6:10002', 'chr14:11909'], ['chr12:30122'], ['chr4:100351'], ['chr1:1']]
